In [3]:
import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
import segmentation_models_pytorch as smp

from src.datasets.INRIAAerialImageLabellingDataset import (
    INRIAAerialImageLabellingDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

d:\__repos\aerial_segmentation\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42

In [7]:
INRIA_DATASET_PATH = "data/INRIAAerialImageLabellingDataset"

In [8]:
labeled_dataset = INRIAAerialImageLabellingDataset(INRIA_DATASET_PATH, transforms=[ResizeToDivisibleBy32()])
print(len(labeled_dataset))

data/INRIAAerialImageLabellingDataset\train
180


In [9]:
test_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH, split="test", transforms=[ResizeToDivisibleBy32()]
)
print(len(test_dataset))

data/INRIAAerialImageLabellingDataset\test
144


In [10]:
# train_indices, val_indices = train_test_split(
#     range(len(labeled_dataset)),
#     test_size=VAL_SIZE,
#     random_state=SEED
# )

# print(f"Number of training samples: {len(train_indices)}")
# print(f"Number of validation samples: {len(val_indices)}")


In [11]:
# train_split = Subset(labeled_dataset, train_indices)
# print(len(train_split))
# val_split = Subset(labeled_dataset, val_indices)
# print(len(val_split))

In [12]:
# train_loader = DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_split, batch_size=BATCH_SIZE, shuffle=True)

train_loader = DataLoader(labeled_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [13]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 5024, 5024])
torch.Size([1, 1, 5024, 5024])


In [14]:
to_pil_transform = transforms.ToPILImage()
img = to_pil_transform(images.squeeze())

In [15]:
# img.show()

In [16]:
print(type(masks))
print(masks.shape)
print(masks.max())
print(masks.min())
unique, counts = np.unique(masks, return_counts=True)
print(unique)
print(counts)

<class 'torch.Tensor'>
torch.Size([1, 1, 5024, 5024])
tensor(1.)
tensor(0.)
[0.0000000e+00 4.4554472e-06 6.4373016e-06 ... 9.9998677e-01 9.9998891e-01
 1.0000000e+00]
[23095741        1        1 ...        1        1  1929884]


In [17]:
msk = to_pil_transform(masks.squeeze())

In [18]:
# msk.show()

In [19]:
model = smp.Unet(
    encoder_name="resnet34",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,  # model output channels (number of classes in your dataset)
).to(device)

In [20]:
model.eval()

with torch.no_grad():
    for i, (images, masks) in enumerate(train_loader):

        cumulated_iou_score = 0.0
        height = images.shape[2]
        width = images.shape[3]

        # if height % 32 == 0 and width % 32 == 0:
        #     print("Height and width are divisible by 32")
        # else:
        #     print("Height and width are not divisible by 32")


        output = model(images.to(device))
        output = torch.softmax(output, dim=1)
        class_output = torch.argmax(output, dim=1)
        
        # print("output")
        # print(type(output))
        # print(output.shape)
        # print(output.max())
        # print(output.min())
        # print()
        # print("masks")
        # print(type(masks))
        # print(masks.shape)
        # print(masks.max())
        # print(masks.min())
        # print()
        # print("class_output")
        # print(type(class_output))
        # print(class_output.shape)
        # print(class_output.max())
        # print(class_output.min())
        # print()

        # images = to_pil_transform(images.squeeze())
        # images.show()
        # images_array = np.array(images)

        # Show the images using matplotlib
        # plt.imshow(images_array, cmap='gray')
        # plt.axis('off')

        # output = to_pil_transform(output.squeeze())
        # output.show()
        # output_array = np.array(output.squeeze())

        # Show the output using matplotlib
        # plt.figure()
        # plt.imshow(output_array, cmap='gray')
        # plt.axis('off')
        # plt.show()

        # class_output_array = np.array(class_output.squeeze())

        # masks_pil= to_pil_transform(masks.squeeze())
        # masks.show()
        # Convert the PIL Image to a numpy array
        # mask_array = np.array(masks_pil)

        # Show the mask using matplotlib
        # plt.imshow(mask_array, cmap='gray')
        # plt.axis('off')
        # plt.show()

        tp, fp, fn, tn = smp.metrics.get_stats(
            class_output.squeeze().to("cpu"), masks.int().squeeze(), mode="binary"
        )
        # print(f"TP: {tp}")
        # print(f"FP: {fp}")
        # print(f"TN: {tn}")
        # print(f"FN: {fn}")
        
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        print(f"IoU score: {iou_score}")

        cumulated_iou_score += iou_score
        del images, masks, output, class_output
        gc.collect()
        torch.cuda.empty_cache()

        if i >= 15:
            break

average_iou_score = cumulated_iou_score / i
print(f"Average IoU score: {average_iou_score}")

IoU score: 0.05388328433036804
IoU score: 0.06953888386487961
IoU score: 0.027721716091036797
IoU score: 0.03554130345582962
IoU score: 0.08223626017570496
IoU score: 0.0024121578317135572
IoU score: 0.03919782117009163
IoU score: 0.06693419814109802
IoU score: 0.057004570960998535
IoU score: 0.012791343964636326
IoU score: 0.05835757404565811
IoU score: 0.07981478422880173
IoU score: 0.08139630407094955
IoU score: 0.008539685979485512
IoU score: 0.0305361058562994
IoU score: 0.08729837089776993
Average IoU score: 0.005819891579449177


In [ ]:
plt.imshow(images_array, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(mask_array, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
# plt.imshow(output_array, cmap='gray')
# plt.axis('off')
# plt.show()
plt.matshow(output_array[0, :, :])
plt.matshow(output_array[1, :, :])

In [ ]:
plt.imshow(class_output_array, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
plt.matshow(class_output_array)

In [ ]:
class_output_array

In [ ]:
class_output_array.max()

In [ ]:
class_output_array.min()

In [ ]:
unique, counts = np.unique(class_output_array, return_counts=True)
print(dict(zip(unique, counts)))